<br> <font color = black size=7>Applied C.F Assignment 1</font> <br>

In [208]:
import numpy as np
import pandas as pd

<br> <font color = black size=5>Introduction</font> <br>

The aim of this report is to solve the system of equations **Ax = b** with Python using different methods. We will apply two different **LU** decomposition techniques: Doolittle's Method and Crout's Method to then solve the system of equations using forward and backward substitution. Also we will apply Gauss-Seidel and SOR iterative techniques and experiment with different error values to observe how fast each of them converges.

# Strictly Diagonally Dominant Test

A **strictly diagonally dominant** matrix **M** is a square matrix that for each row, the absolute value of the term that lies in the leading diagonal is greater than the sum of the absolute values of the remaining terms of that row.
\begin{gather}
M = 
\begin{bmatrix}
a & b & c\\
d & e & f\\
g & h & i
\end{bmatrix}
\end{gather}

$\mid a \mid  >  \mid b \mid + \mid c \mid$ and $\mid e \mid  >  \mid d \mid + \mid f \mid$ and $\mid  i \mid  >  \mid g \mid + \mid h \mid $


When a matrix is **strictly diagonally dominant** it means that it can be inverted. This guarantees convergence when using iterative techniques to solve the system of equations in matrix **M**. <p> We built a function that returns True if the matrix M is strictly diagonally dominant, False otherwise.



In [209]:
def diagonallyDom(M):
    m = len(M)
    for i in range(m):
        D = np.absolute(M[i][i])
        if D == 0:
            return False
        suma = 0
        for j in range(m):
            if i != j:
                suma = suma + np.absolute(M[i][j])
        if D <= suma:
            return False
            
    return True

Below we have a function that tries to convert a matrix M to a strictly diagonally dominant matrix. The function takes the index of the largest absolute value in each row, and reorders that matrix by putting that row in the index taken. This function only modifies rows, it will return that reordered matrix and the reordered vector b.

In [210]:
def tryDiagonally(M,b):
    if diagonallyDom(M):
        return M, b
    
    s = len(M)
    A = np.zeros((s,s))
    y = np.zeros(s)
    
    maximum = 0
    index = 0
    for i in range(s):
        maximum = 0
        index = 0
        for j in range(s):
            if np.absolute(M[i][j]) > maximum:
                maximum = np.absolute(M[i][j])
                index = j
        A[index] = M[i]
        y[index] = b[i]
    
    return A,y
            

## Triangular Matrix

The following function evaluates if **M** is a triangular matrix. It returns a 1 if it is a lower triangular matrix, a 2 if it is an upper triangular matrix and 0 otherwise. Example of an upper triangular matrix:

\begin{gather}
M = 
\begin{bmatrix}
3 & 1 & 7\\
0 & 5 & 3\\
0 & 0 & 6
\end{bmatrix}
\end{gather}

In [211]:
def triangular(M):
    n = len(M)
    if n != len(M[0]):
        return 0
    upper = True
    for i in range(n):
        for j in range(i):
            if np.round(M[i][j], 3) !=0:
                upper = False
    lower = True
    for i in range(n):
        a = n-i-1
        for j in range(a):
            b = n-j-1
            if np.round(M[i][b], 3) !=0:
                lower = False
    if lower:
        return 1
    elif upper:
        return 2
    else:
        return 0
    

# LU Decomposition

We can write a square **nxn** matrix **A** as a product of two matrices **L** and **U**, **A = LU**. Where **L** is a lower triangular matrix and **U** and upper triangular matrix. By doing this factorization, the problem of solving the system of equations **Ax = b** becomes much easier, we can solve it by applying a forward substitution and a backward substitution.

## Doolittle's Method

**Doolittle's Method** is a type of **LU** decomposition. Gaussian Elimination is applied to **A** in order to get **U**. In this method, **L** is a unit lower triangular matrix, meaning that every term in its leading diagonal is a 1. The entries that go below the leading diagonal of the **L** matrix are the multipliers in the Gaussian Elimination process. 

The following function decomposes the matrix **A** into **L** and **U** using **Doolittle's Method**

In [212]:
def Doolittle(M):
    M = M*1.0
    
    s = len(M)
    
    I = np.identity(len(M))

    for h in range(s-1):
        for i in range(h,s-1):
            if M[i+1][h] != 0:
                num = float(M[i+1][h]/M[h][h])
                I[i+1][h] = num
                for j in range(s):
                    temp = M[i+1][j]
                    M[i+1][j] = temp - (num*M[h][j])
    return I,M

## Crout's Method

**Crout's Method** is another type of **LU** decomposition. In this method, the upper triangular matrix **U** has the unit leading diagonal, differing from **Doolitle's Method** where the unit leading diagonal is in the lower triangular matrix **L**.

The following function decomposes the matrix **A** into **L** and **U** using **Crout's Method**

In [213]:
def Crout(M):
    a = len(M)
    L = np.zeros([a,a])
    U = np.identity(a)

    for i in range(a):
        for j in range(i+1):
            for h in range(j+1):
                if h == 0:
                    L[i][j] = M[i][j]
                else:
                    L[i][j] = L[i][j] -U[h-1][j]*L[i][h-1]

        for m in range(a-1-i):
            for n in range(i+1):
                if n == 0:
                    U[i][m+1+i] = M[i][m+1+i]/L[i][i]
                else:
                    U[i][m+1+i] = U[i][m+1+i] -(U[n-1][m+1+i]*L[i][n-1]/L[i][i])
    return L,U

## Solving Ax = b using LU Decomposition

As we mentioned before, when we decompose **A** into **L** and **U** it is very easy to solve the system **Ax = b** using forward and backward substitutions.

We have that **Ax =(LU)x = L(Ux) = b**. We can devide this into two smaller problems.

**1. Lz = b**

**2. Ux = z**.

First we solve for **z** in **1** using **forward substitution** and then, using **z**, we solve for **x** in **2** **using backward** substitution. The following **solver** function uses the algorithm mentioned above to solve the system of equations. It takes as inputs **L**,**U** and **b** and returns **x**. It also verifies that the inputs **L** and **U** are lower and upper triangular matrices respectively.


In [214]:
def solver(L,U,b):
    # Forward substitution
    if triangular(L) != 1:
        return "L is not a lower triangular matrix"
    n = len(b)
    z = np.zeros(n)
    for i in range(n):
        z[i] = b[i]
        for j in range(i):
            z[i] = z[i]-L[i][j]*z[j]    
        z[i] = z[i]/L[i][i]
    print("z = ", z)
    
    if triangular(U) != 2:
        return "U is not an upper triangular matrix"
    
    # Backward substitution
    y = np.zeros(n)
    for i in range(n):
        a = n-1-i
        y[a] = z[a]
        for j in range(i):
            b = n-1-j
            y[a] = y[a]-U[a][b]*y[b]    
        y[a] = y[a]/U[a][a]
    print("x = ",y)
    return y
    
    
        

# Gauss-Seidel Method

The **Gauss Seidel Method** is an iterative method that is used solve linear systems of equations. Convergence is guaranteed when the matrix is strictly diagonally dominant. The following function applies this method to solve the system of equations **Ax = b**. It takes as inputs a matrix, a vector **b** and an initial guess **x0**, returning the resulting **x** after the final iteration, and the number of iterations. This function iterates until it reaches consistency to the tolerance $\varepsilon$ using  the **infinity norm** as convergence criteria. The function also takes a boolean pr parameter that is used only to indicate if we want to print the results.

**Infinity Norm**

$l_\infty = \frac{\mid \mid \vec{x}^{(k+1)}-\vec{x}^{(k)}\mid \mid_\infty}{\mid \mid \vec{x}^{(k)}\mid \mid_\infty} < \varepsilon$\\

$\varepsilon$ is the tolerance > 0.

$\mid \mid \vec{x} \mid \mid_\infty = max \mid x_i \mid $ 

$1\leq i \leq n$


In [215]:
def GS(M,x,p,pr):
    prev = np.zeros((2,len(M)),float)

    m = len(M)

    inf_norm = 1
    count = 0
    while inf_norm > p :
        for i in range(m):
            prov = b[i]
            for j in range(m):
                if i != j:
                    prov = prov -M[i][j]*x[j]
            x[i] = prov/M[i][i]

        prev[1] = prev[0]
        prev[0] = x

        if count != 0:
            inf_norm = (max(np.absolute(prev[0]-prev[1])))/max(np.absolute(prev[1]))
            if pr:
                print("error = ",inf_norm)
        count = count +1
        if pr:
            print(x)
            print("-----------------")
    return x, count

# Successive-Over Relaxation (SOR) Method

This is a technique that can be applied to iterative methods such as the **Gauss-Seidel Method**. The aim of the **SOR** method is to introduce an acceleration factor **w** to speed up convergence where **1** **<** **w** **<** **2**.

The following function applies **SOR** to the **Gauss-Seidel Method** to solve the system of equations **Ax = b**. It takes as inputs a matrix, a vector **b**,initial guess **x0**, and the acceleration factor **w** returning the resulting **x** after the final iteration. This function iterates until it reaches consistency to the tolerance $\varepsilon$ using  the **infinity norm** as convergence criteria. Again, the function also takes a boolean pr parameter that is used only to indicate if we want to print the results.

In [216]:
def SOR(M,x,w,p,pr):
    
    prev = np.zeros((2,len(M)),float)

    m = len(M)

    inf_norm = 1
    count = 0
    while inf_norm > p:
        for i in range(m):
            prov = b[i]
            for j in range(m):
                if i != j:
                    prov = prov -M[i][j]*x[j]
            pre = x[i]
            x[i] = ((1-w)*pre)+(w*prov/M[i][i])

        prev[1] = prev[0]
        prev[0] = x

        if count != 0:
            inf_norm = (max(np.absolute(prev[0]-prev[1])))/max(np.absolute(prev[1]))
            if pr:
                print("error = ",inf_norm)
        count = count +1
        if pr:
            print(x)
            print("-----------------")
    return x, count

# Results

We begin by initializing the system of equations with the matrix **A** and the vector **b**
We will solve **Ax = b** using the methods mentioned earlier

In [217]:
A = np.array([[0,3,-1,8], 
              [-1,11,-1,3],
              [2,-1,10,-1],
              [10,-1,2,0]])

b = np.array([15,25,-11,6])

## Strictly Diagonally Dominant Test

We will test if our Matrix **A** is strictly diagonally dominant using the function **diagonallyDom(M)** we defined earlier

In [218]:
diagonallyDom(A)

False

We got a **False** result,  this means that our matrix **A** is not strictly diagonally dominant, we will use our previously defined **tryDiagonally()** function to rearrange the rows in matrix **A** and see if we can convert it into a strictly diagonally dominant matrix.

In [219]:
A, b = tryDiagonally(A,b)
print("A = ")
print(A)
print("b = ")
print(b)
diagonallyDom(A)

A = 
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
b = 
[  6.  25. -11.  15.]


True

We can see above that matrix **A** and vector **b** have been rearranged and tested for with our **diagonallyDom()**. We can see that it returns **True**, this means that matrix **A** became indeed a strictly diagonally dominant matrix. We will work with this new modified **A** matrix and **b** vector from now on. Once we transformed our matrix **A** into a strictly diagonally dominant matrix, we proceeded to decompose it into **L** and **U** using **Doolittle's** and **Crout's** method

## LU Decomposition

### Doolittle

In [220]:
L,U = Doolittle(A)
print("L = ")
print(np.round(L, 3))

print("U = ")
print(np.round(U, 3))

L = 
[[ 1.     0.     0.     0.   ]
 [-0.1    1.     0.     0.   ]
 [ 0.2   -0.073  1.     0.   ]
 [ 0.     0.275 -0.082  1.   ]]
U = 
[[10.    -1.     2.     0.   ]
 [ 0.    10.9   -0.8    3.   ]
 [ 0.     0.     9.541 -0.78 ]
 [ 0.    -0.     0.     7.111]]


From **Doolittle's** method we can see that our resulting **L** and **U** matrices are indeed lower and upper triangular matrices respectively, with **L** containing a unit leading diagonal.

In [221]:
L.dot(U)

array([[10., -1.,  2.,  0.],
       [-1., 11., -1.,  3.],
       [ 2., -1., 10., -1.],
       [ 0.,  3., -1.,  8.]])

We verified that our resulting matrices **L** and **U** were correct by doing a matrix multiplication between them and verifying that the result was **A**, we can see above that **LU = A** .

Then we use our **solver** function to calculate the intermediate vector **z** in [**Ax =(LU)x = L(Ux) = b**] and the final solution **x** using forward and backward substitution.

In [222]:
solver(L,U,b)

z =  [  6.          25.6        -10.32110092   7.11057692]
x =  [ 1.  2. -1.  1.]


array([ 1.,  2., -1.,  1.])

In [223]:
solution = np.linalg.inv(A).dot(b)
print(solution)

[ 1.  2. -1.  1.]


Above, we can see that using our solver function we got a result of **x = [1,2,-1,1]**, we verified that this solution is correct using **x = inv(A)b**, we can see that it returns the same result.

Now we move forward to do the same process with **Crout's method**

### Crout

In [224]:
L,U = Crout(A)
print(np.round(L, 3))
print(np.round(U, 3))

[[10.     0.     0.     0.   ]
 [-1.    10.9    0.     0.   ]
 [ 2.    -0.8    9.541  0.   ]
 [ 0.     3.    -0.78   7.111]]
[[ 1.    -0.1    0.2    0.   ]
 [ 0.     1.    -0.073  0.275]
 [ 0.     0.     1.    -0.082]
 [ 0.     0.     0.     1.   ]]


In [225]:
solver(L,U,b)

z =  [ 0.6         2.34862385 -1.08173077  1.        ]
x =  [ 1.  2. -1.  1.]


array([ 1.,  2., -1.,  1.])

We can observe that in this method, we get the unit leading diagonal in the **U** matrix instead of in the **L** matrix as in Doolittle's method,but when we apply our solver function using forward and backward substitution we still get the correct result of **x = [1,2,-1,1]**

## Gauss-Seidel Method

We solved the system of equations **Ax = b** using the GS function explained above that computes the Gauss-Seidel Method. Since we verified that our matrix **A** is strictly diagonally dominant, we know this method will converge to the actual solution. We will iterate until the method reaches consistency to the tolerance $\varepsilon$ using  the **infinity norm** as convergence criteria.

In this first part we begin with an initial guess of **x0 = [0,0,0,0]**

In [181]:
e = 0.0001
x0 = np.zeros(len(A),float)
pr = True
x, it = GS(A,x0,e,pr)
print("Number of iterations = ", it)

[ 0.6         2.32727273 -0.98727273  0.87886364]
-----------------
error =  0.18484375000000003
[ 1.03018182  2.03693802 -1.0144562   0.98434122]
-----------------
error =  0.016388814658793216
[ 1.00658504  2.00355502 -1.00252738  0.99835095]
-----------------
error =  0.002856953090344185
[ 1.00086098  2.00029825 -1.00030728  0.99984975]
-----------------
error =  0.0003847917873479008
[ 1.00009128  2.00002134 -1.00003115  0.9999881 ]
-----------------
error =  4.1457869928006095e-05
[ 1.00000836  2.00000117 -1.00000275  0.99999922]
-----------------
Number of iterations =  6


Now we use an initial guess of **x0 = [1,1,1,1]**

In [182]:
e = 0.0001
x0 = np.ones(len(A),float)
pr = True
x, it = GS(A,x0,e,pr)
print("Number of iterations = ", it)

[ 0.5         2.13636364 -0.88636364  0.96306818]
-----------------
error =  0.22978723404255322
[ 0.99090909  2.01957645 -0.99991736  0.99266916]
-----------------
error =  0.008612275598771455
[ 1.00194112  2.0021833  -1.00090298  0.99906839]
-----------------
error =  0.0009864457246363382
[ 1.00039893  2.00020825 -1.00015212  0.99990289]
-----------------
error =  0.0001738197994263564
[ 1.00005125  2.00001731 -1.00001823  0.99999123]
-----------------
error =  2.293582118326794e-05
[ 1.00000538  2.00000122 -1.00000183  0.99999931]
-----------------
Number of iterations =  6


Comparing the number of iteration it took for the system to converge using the initial guesses of **x0 = [0,0,0,0]** and **x0 = [1,1,1,1]**, we can observe that they both took 6 iterations to converge with consistency of 4 decimal places. If we analyze the error in each iteration, we can observe that using **x0 = [1,1,1,1]** helped reduce the error faster. Using **x0 = [0,0,0,0]** we have a final error of 4.14e-5 compared to 2.29e-5 using **x0 = [1,1,1,1]**.

We will try using a consistency of 5, 6 and 7 decimal places to see if using **x0 = [1,1,1,1]** makes the method converge with lower iterations than using **x0 = [0,0,0,0]**.

In [226]:
e = 0.0000001
x0 = np.zeros(len(A),float)
pr = True
x, it = GS(A,x0,e,pr)
print("Number of iterations = ", it)

[ 0.6         2.32727273 -0.98727273  0.87886364]
-----------------
error =  0.18484375000000003
[ 1.03018182  2.03693802 -1.0144562   0.98434122]
-----------------
error =  0.016388814658793216
[ 1.00658504  2.00355502 -1.00252738  0.99835095]
-----------------
error =  0.002856953090344185
[ 1.00086098  2.00029825 -1.00030728  0.99984975]
-----------------
error =  0.0003847917873479008
[ 1.00009128  2.00002134 -1.00003115  0.9999881 ]
-----------------
error =  4.1457869928006095e-05
[ 1.00000836  2.00000117 -1.00000275  0.99999922]
-----------------
error =  3.848654328653028e-06
[ 1.00000067  2.00000002 -1.00000021  0.99999996]
-----------------
error =  3.110314673072606e-07
[ 1.00000004  1.99999999 -1.00000001  1.        ]
-----------------
error =  2.1116799365537246e-08
[ 1.  2. -1.  1.]
-----------------
Number of iterations =  9


In [227]:
e = 0.0000001
x0 = np.ones(len(A),float)
pr = True
x, it = GS(A,x0,e, pr)
print("Number of iterations = ", it)

[ 0.5         2.13636364 -0.88636364  0.96306818]
-----------------
error =  0.22978723404255322
[ 0.99090909  2.01957645 -0.99991736  0.99266916]
-----------------
error =  0.008612275598771455
[ 1.00194112  2.0021833  -1.00090298  0.99906839]
-----------------
error =  0.0009864457246363382
[ 1.00039893  2.00020825 -1.00015212  0.99990289]
-----------------
error =  0.0001738197994263564
[ 1.00005125  2.00001731 -1.00001823  0.99999123]
-----------------
error =  2.293582118326794e-05
[ 1.00000538  2.00000122 -1.00000183  0.99999931]
-----------------
error =  2.444438770033308e-06
[ 1.00000049  2.00000007 -1.00000016  0.99999996]
-----------------
error =  2.249473141715138e-07
[ 1.00000004  2.         -1.00000001  1.        ]
-----------------
error =  1.8007612280072827e-08
[ 1.  2. -1.  1.]
-----------------
Number of iterations =  9


After using lower errors to test the convergence of the **Gauss-Seidel method** we observe that even though using **x0 = [1,1,1,1]** produces slightly lower error than using **x0 = [0,0,0,0]**, the number of iterations for convergence up to 4,5,6 and 7 decimal places does not change. Above we show the example with consistency up to 7 decimal places and we see that in both cases it took 9 iteration for the method to converge.

## SOR Method

Now, we will use our **SOR** function described above to solve the **Ax = b**. We will use an acceleration factor **w = 1.1** to test if the system converges faster than the regular Gauss-Seidel method. Again, we will iterate until the method reaches consistency to the tolerance $\varepsilon$ using  the **infinity norm** as convergence criteria.We will use **x0 = [0,0,0,0]**.

In [185]:
e = 0.0001
w = 1.1
g = np.zeros(len(A),float) 
pr = True
x, it = SOR(A,g,w,e,pr)
print("Number of iterations = ", it)

[ 0.66        2.566      -1.07294     0.85649575]
-----------------
error =  0.22432269875292288
[ 1.1123068   1.99038796 -1.03425629  1.01360515]
-----------------
error =  0.05881186187694755
[ 0.99524838  1.99297887 -0.99480477  1.00225005]
-----------------
error =  0.0037249485598974
[ 0.99855989  2.00040261 -0.99991091  0.99962117]
-----------------
error =  0.0008042432684686864
[ 1.0001687   2.00009917 -1.00007679  0.99998642]
-----------------
error =  7.887918876190647e-05
[ 1.00001093  1.99998757 -0.99999759  1.00000682]
-----------------
Number of iterations =  6


From the result above, we can observe that the number of iterations to reach convergence (6) were the same as when using the regular Gauss-Seidel method. Also we can noticed that the final error was higher with 7.88e-5 compared to 4.14e-5 when using the regular Gauss-Seidel method with **x0 = [0,0,0,0]**. This means that in this case, introducing the acceleration factor is actually harming the iterative method by incrementing the error. We actually tested different acceleration factors between 1 and 2 but the results where even worse resulting in more iterations (See example below with **w = 1.3**) .

In [186]:
e = 0.0001
w = 1.3
g = np.zeros(len(A),float)
pr = True
x, it = SOR(A,g,w,e,pr)
print("Number of iterations = ", it)

[ 0.78        3.04672727 -1.23672545  0.75125257]
-----------------
error =  0.4166353760843501
[ 1.26362316  1.77735291 -1.05880567  1.17360876]
-----------------
error =  0.20050309496097163
[ 0.9072584   1.9873318  -0.93732321  0.9642781 ]
-----------------
error =  0.051637699664571
[ 1.00987965  2.02504035 -1.02276035  0.99481084]
-----------------
error =  0.01613238325660599
[ 1.00620904  1.99237162 -0.99645253  1.00585205]
-----------------
error =  0.005012009929900946
[ 0.99622326  2.0001866  -0.99929726  0.99826761]
-----------------
error =  0.002375435885586547
[ 1.00097457  2.00075646 -1.00059108  1.00005489]
-----------------
error =  0.0005385359943596845
[ 0.99995965  1.99967898 -0.99984678  1.00016493]
-----------------
error =  0.0001844061145055497
[ 0.99993054  2.00004773 -1.00000026  0.99992721]
-----------------
error =  4.8286884791538394e-05
[ 1.00002711  2.00001466 -1.00001453  1.00001233]
-----------------
Number of iterations =  10


To conclude our report, we will loop over 10 different tolerances $\varepsilon$, ranging from 1e-2 to 1e-11 and **w = 1.1**. In each step we will apply the **Gauss-Seidel method** using **x0 = [0,0,0,0]** and **x0 = [1,1,1,1]**, and SOR using **x0 = [0,0,0,0]**. We will report how many iterations it took each method to converge for each $\varepsilon$ and comment on the results.

In [206]:
results = np.zeros((10,4))
w = 1.1
e = 0.01
pr = False
for i in range(10):
    x0_1 = np.zeros(len(A),float)
    x0_2 = np.ones(len(A),float)
    x0_3 = np.zeros(len(A),float)
    
    results[i][0] = e
    
    x, it_1 = GS(A,x0_1,e,pr)
    results[i][1] = it_1
    
    x, it_2 = GS(A,x0_2,e,pr)
    results[i][2] = it_2
    
    x, it_3 = SOR(A,x0_3,w,e,pr)
    results[i][3] = it_3
    
    e = e/10


In [207]:
df = pd.DataFrame(results,columns =["epsilon", "G-S x0 = zeros|", "G-S x0 = ones|", "SOR x0 = zeros"])
df = df.set_index('epsilon')
df["G-S x0 = zeros|"] = df["G-S x0 = zeros|"].astype(int)
df["G-S x0 = ones|"] = df["G-S x0 = ones|"].astype(int)
df["SOR x0 = zeros"] = df["SOR x0 = zeros"].astype(int)
print("Number of Iterations to reach Convergence")
print(df)

Number of Iterations to reach Convergence
              G-S x0 = zeros|  G-S x0 = ones|  SOR x0 = zeros
epsilon                                                      
1.000000e-02                4               3               4
1.000000e-03                5               4               5
1.000000e-04                6               6               6
1.000000e-05                7               7               7
1.000000e-06                8               8               9
1.000000e-07                9               9              10
1.000000e-08               10              10              11
1.000000e-09               11              11              12
1.000000e-10               11              11              13
1.000000e-11               12              12              14


In the table above we can observe that the best method was the **Gauss-Seidel** intialized in **x0 = [1,1,1,1]**. In every case it had equal or lower iterations than the **Gauss-Seidel** intialized in **x0 = [0,0,0,0]** and the **SOR** method. It was surprising that the **SOR** method performed the worst since the whole purpose of introducing an over-relaxed acceleration factor was to accelerate convergence. Further analysis has to be made in order to evaluate why this acceleration factor of **w= 1.1** is not helping the system to converge faster.